<a href="https://colab.research.google.com/github/kunai-3txk/Prediction-of-seriously-ill-patients/blob/main/BaseLine_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
 
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd

#config

In [3]:
class config:
    RANDOM_STATE=100  
    FRAC = 0.2
    N_FOLD = 5
    N_BAGGING = 3
    UNDER_SAMPLING = True
    
class paths:
    # kaggle環境ならTrue
    if 'KAGGLE_URL_BASE' in set(os.environ.keys()):
        common_path = "/kaggle/input/amp-parkinsons-disease-progression-prediction"
    
    # colaboratory環境ならTrue
    if 'COLAB_GPU' in set(os.environ.keys()):
        common_path = "/content/drive/MyDrive/ill"
    
    SUBMISSION = common_path + "/submission.csv"
    TRAIN = common_path + "/train_df.csv"
    TEST = common_path + "/test_df.csv"
    SUB_OUT = common_path + "/sub/"
     

#Load Data

In [4]:
train = pd.read_csv(paths.TRAIN)
print('train shape:',train.shape)

test = pd.read_csv(paths.TEST)
print('test shape:',test.shape)

submission = pd.read_csv(paths.SUBMISSION)
print('submission shape:',submission.shape)

train shape: (51359, 85)
test shape: (12840, 84)
submission shape: (12840, 2)


## preprocessing

In [5]:
num_cols = ['age', 'bmi', 'height', 'weight', 'icu_4', 'icu_6'
            ,'glasgow_coma_scale_1', 'glasgow_coma_scale_2','glasgow_coma_scale_3', 'glasgow_coma_scale_4'
            , 'heart_rate', 'blood_oxy', 'arterial_pressure',
            'respiratory_rate', 'temp'
            , 'blood_pressure_1', 'blood_pressure_2', 'blood_pressure_3', 'blood_pressure_4',
            'v1_heartrate_max'
            , 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14','v15', 'v16'
            , 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w10', 'w11', 'w12', 'w13', 'w14','w15', 'w16', 'w17', 'w18'
            , 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']


X_train = train[num_cols]
X_train = X_train.fillna(-1)
y_train = train['target_label']

test_X = test[num_cols]
test_X = test_X.fillna(-1)

#NN

In [13]:
train_X, val_X, train_y, val_y = train_test_split(X_train.values, y_train.values, test_size = 0.25, random_state=0)

train_X = torch.from_numpy(train_X).float()
train_y = torch.from_numpy(train_y).long()
val_X = torch.from_numpy(val_X).float()
val_y = torch.from_numpy(val_y).long()

test_X = torch.from_numpy(test_X.values).float()

train_td=TensorDataset(train_X, train_y)

train_loader = DataLoader(train_td, batch_size=15, shuffle=True)

In [7]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.fc1 = nn.Linear(59, 128) #入力層59個（特徴量分）、中間層を128に設定
        self.fc2 = nn.Linear(128, 2)  #入力層が前の中間層の128となり、出力層は２分類ですので、2と設定
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc1 = nn.Linear(59, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

model = Net2()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(180): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 665.649908836931
120 622.3198429457843
180 623.3581464551389


In [21]:
val_X, val_y = Variable(val_X), Variable(val_y)
result = torch.max(model(val_X).data, 1)[1]
accuracy = sum(val_y.data.numpy() == result.numpy()) / len(val_y.data.numpy())
print(accuracy)

#0.9129283489096574
#0.9130841121495327

result_test = torch.max(model(test_X).data,1)[1].numpy()

array([0, 0, 0, ..., 0, 0, 0])

#1D-CNN
https://qiita.com/tatsuya11bbs/items/7a419568611157081f1d

In [25]:
class Net1D(nn.Module):
  def __init__(self):
    super(Net1D,self).__init__()

    self.conv1 = nn.Conv1d(1, 8,kernel_size=3, stride=1)
    self.bn1 = nn.BatchNorm1d(8)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2)

    self.conv2 = nn.Conv1d(8, 16,kernel_size=3, stride=1)
    self.bn2 = nn.BatchNorm1d(16)
    self.conv3 = nn.Conv1d(16,64,kernel_size=3, stride=1)
    self.gap = nn.AdaptiveAvgPool1d(1)
    self.fc = nn.Linear(64,2)


  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv3(x)
    x = self.gap(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)

    return x

model = Net1D()
in_data = torch.randn(8,1,50)
out_data = model(in_data)
print(out_data.size()) #torch.Size([8, 2])

torch.Size([8, 2])


#2D-CNN

In [27]:
class Net2D(nn.Module):
  def __init__(self):
    super(Net2D,self).__init__()

    self.conv1 = nn.Conv2d(3,16,kernel_size=3,stride=2)
    self.bn1 = nn.BatchNorm2d(16)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,32,kernel_size=3,stride=2)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32,64,kernel_size=3,stride=2)
    self.gap = nn.AdaptiveAvgPool2d(1)
    self.fc1 = nn.Linear(64,32)
    self.fc2 = nn.Linear(32,2)


  def forward(self,x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.conv3(x)
    x = self.gap(x)
    x = x.view(x.size(0),-1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

model = Net2D()
in_data = torch.randn(1,16,3,50)
out_data = model(in_data)
print(out_data.size()) #torch.Size([8, 2])

RuntimeError: ignored